In [66]:
# Calibration of camera
import numpy as np
import cv2
import glob
import pickle

objp = np.zeros((6*9,3), np.float32)
objp[:,:2] = np.mgrid[0:9,0:6].T.reshape(-1,2)

objpoints = []
imgpoints = []

images = glob.glob("./camera_cal/calibration*.jpg")

for idx, fname in enumerate(images):
    img = cv2.imread(fname)
    gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
    
    ret, corners = cv2.findChessboardCorners(gray, (9,6), None)
    
    if ret:
        print("Chessboard corners of file: {}".format(fname))
        objpoints.append(objp)
        imgpoints.append(corners)
        
        
        cv2.drawChessboardCorners(img, (9,6), corners, ret)
        ofname = "output_images/with_corners{}.jpg".format(idx)
        cv2.imwrite(ofname, img)
        
img = cv2.imread("./camera_cal/calibration1.jpg")  # To get the image shape
img_size = (img.shape[1], img.shape[0])
ret, mtx, dist, rvecs, tvecs = cv2.calibrateCamera(objpoints, imgpoints, img_size, None, None)

dist_mtx = {}
dist_mtx["mtx"] = mtx
dist_mtx["dist"] = dist
pickle.dump(dist_mtx, open("./camera_cal/calibration_pickle.p", "wb"))

Chessboard corners of file: ./camera_cal/calibration10.jpg
Chessboard corners of file: ./camera_cal/calibration11.jpg
Chessboard corners of file: ./camera_cal/calibration12.jpg
Chessboard corners of file: ./camera_cal/calibration13.jpg
Chessboard corners of file: ./camera_cal/calibration14.jpg
Chessboard corners of file: ./camera_cal/calibration15.jpg
Chessboard corners of file: ./camera_cal/calibration16.jpg
Chessboard corners of file: ./camera_cal/calibration17.jpg
Chessboard corners of file: ./camera_cal/calibration18.jpg
Chessboard corners of file: ./camera_cal/calibration19.jpg
Chessboard corners of file: ./camera_cal/calibration2.jpg
Chessboard corners of file: ./camera_cal/calibration20.jpg
Chessboard corners of file: ./camera_cal/calibration3.jpg
Chessboard corners of file: ./camera_cal/calibration6.jpg
Chessboard corners of file: ./camera_cal/calibration7.jpg
Chessboard corners of file: ./camera_cal/calibration8.jpg
Chessboard corners of file: ./camera_cal/calibration9.jpg


In [67]:
# tracker
class Tracker:
    def __init__(self, window_w, window_h, margin, ym=1, xm=1, smooth_factor=15):
        self.window_w = window_w
        self.window_h = window_h
        self.margin = margin
        self.ym_per_pixel = ym
        self.xm_per_pixel = xm
        self.smooth_factor = smooth_factor
        self.recent_centers = []
        
    def find_window_centroids(self, warped):
        window_w = self.window_w
        window_h = self.window_h
        margin = self.margin
        
        window_centroids= []
        window = np.ones(window_w)
        
        l_sum = np.sum(warped[int(3*warped.shape[0]/4):, :int(warped.shape[1]/2)], axis = 0)
        l_center = np.argmax(np.convolve(window, l_sum))-window_w/2
        r_sum = np.sum(warped[int(3*warped.shape[0]/4):, int(warped.shape[1]/2):], axis = 0)
        r_center = np.argmax(np.convolve(window, r_sum))-window_w/2+int(warped.shape[1]/2)
        
        window_centroids.append((l_center, r_center))
        
        for level in range(1, int(warped.shape[0]/window_h)):
            image_layer = np.sum(warped[int(warped.shape[0]-(level+1)*window_h):int(warped.shape[0]-level*window_h),:],
                                 axis=0)
            conv_signal = np.convolve(window, image_layer)
            offset = window_w/2
            l_min_index = int(max(l_center+offset-margin, 0))
            l_max_index = int(min(l_center+offset+margin, warped.shape[1]))
            l_center = np.argmax(conv_signal[l_min_index:l_max_index])+l_min_index-offset
            
            r_min_index = int(max(r_center+offset-margin, 0))
            r_max_index = int(min(r_center+offset+margin, warped.shape[1]))
            r_center = np.argmax(conv_signal[r_min_index:r_max_index])+r_min_index-offset
            
            window_centroids.append((l_center, r_center))
        self.recent_centers.append(window_centroids)
        
        return np.average(self.recent_centers[-self.smooth_factor:], axis=0)

In [68]:

        


def abs_sobel_thresh(img, orient='x', sobel_kernel=3, thresh=(0, 255)):
    gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
    # Calculate directional gradient
    if orient == 'x':
        sobel = cv2.Sobel(gray, cv2.CV_64F, 1, 0, ksize=sobel_kernel)
    elif orient == 'y':
        sobel = cv2.Sobel(gray, cv2.CV_64F, 0, 1, ksize=sobel_kernel)
    else:
        raise Error("Unsupported direction for gradient")
    # Apply threshold
    abs_sobel = np.absolute(sobel)
    abs_sobel = np.uint8(255 * abs_sobel / np.max(abs_sobel))
    grad_binary = np.zeros_like(abs_sobel)
    grad_binary[(abs_sobel >= thresh[0]) & (abs_sobel <= thresh[1])] = 1
    return grad_binary

def hls_thresh(img, sthresh=(0, 255), vthresh=(0, 255)):
    hls = cv2.cvtColor(img, cv2.COLOR_RGB2HLS)
    s_channel = hls[:,:,2]
    s_output = np.zeros_like(s_channel)
    s_output[(s_channel > sthresh[0]) & (s_channel <= sthresh[1])] = 1
    
    hsv = cv2.cvtColor(img, cv2.COLOR_RGB2HSV)
    v_channel = hls[:,:,2]
    v_output = np.zeros_like(v_channel)
    v_output[(v_channel > vthresh[0]) & (v_channel <= vthresh[1])] = 1
        
    output = np.zeros_like(s_channel)
    output[(s_output == 1) & (v_output == 1)] = 1
        
    return output

def window_mask(w, h, img_ref, center, level):
    out = np.zeros_like(img_ref)
    out[int(img_ref.shape[0]-(level+1)*h):int(img_ref.shape[0]-level*h),
        max(0,int(center-w)):min(int(center+w), img_ref.shape[1])] = 1
    return out

# TODO: Insert color threasholding based on HSV here

In [85]:



# Undistort test images

mtx = dist_mtx["mtx"]
dist = dist_mtx["dist"]


images = glob.glob("./test_images/test*.jpg")

for idx,fname in enumerate(images):
    img = cv2.imread(fname)
    img = cv2.undistort(img, mtx, dist, None, mtx)
    
    preProcessImage = np.zeros_like(img[:,:,0])
    gradx = abs_sobel_thresh(img, orient='x', thresh=(12, 255))
    grady = abs_sobel_thresh(img, orient='y', thresh=(25, 255))
    c_binary = hls_thresh(img, sthresh=(100, 255), vthresh=(50, 255))
    preProcessImage[(gradx == 1) & (grady == 1) | (c_binary == 1)] = 255
    
    
    
    # Perspective transform area
    img_size = (img.shape[1], img.shape[0])
    bot_width = 0.76
    mid_width = 0.08
    height_pct = 0.62
    bottom_trim = 0.935

    
    src= np.float32([[img.shape[1]*(0.5- mid_width/2), img.shape[0]*height_pct],
                     [img.shape[1]*(0.5+ mid_width/2), img.shape[0]*height_pct],
                     [img.shape[1]*(0.5+bot_width/2), img.shape[0]-bottom_trim],
                    [img.shape[1]*(0.5-bot_width/2), img.shape[0]-bottom_trim]])

    offset = img_size[0]*0.25
        
    dst = np.float32([[offset, 0],
                      [img_size[0]-offset, 0], 
                      [img_size[0]-offset, img_size[1]], 
                      [offset, img_size[1]]])

#     print("src = {}".format(src))
#     print("dst = {}".format(dst))
    
                     
    M = cv2.getPerspectiveTransform(src, dst)
    Minv = cv2.getPerspectiveTransform(dst, src)
    warped = cv2.warpPerspective(preProcessImage, M, img_size, flags=cv2.INTER_LINEAR)
    
    window_w = 25
    window_h = 80
    
    tracker = Tracker(window_w, window_h, margin=25, ym=10/720., xm=4/384., smooth_factor=15)
    xm_per_pix = tracker.xm_per_pixel
    ym_per_pix = tracker.ym_per_pixel
    window_centroids = tracker.find_window_centroids(warped)
    
    l_points = np.zeros_like(warped)
    r_points = np.zeros_like(warped)
    
    leftx = []
    rightx = []
    for level in range(0, len(window_centroids)):
        
        leftx.append(window_centroids[level][0])
        rightx.append(window_centroids[level][1])
        l_mask = window_mask(window_w, window_h, warped, window_centroids[level][0], level)
        r_mask = window_mask(window_w, window_h, warped, window_centroids[level][1], level)
        
        l_points[(l_points == 255) | (l_mask == 1)] = 255
        r_points[(r_points == 255) | (r_mask == 1)] = 255
        
    
    template = np.array(r_points+l_points, np.uint8)
    zero_channel = np.zeros_like(template)
    template = np.array(cv2.merge((zero_channel, template, zero_channel)), np.uint8)
    warpage = np.array(cv2.merge((warped, warped, warped)), np.uint8)
    result = cv2.addWeighted(warpage, 1, template, 0.5, 0.0)
    
    
    yvals = range(0, warped.shape[0])
    res_yvals = np.arange(warped.shape[0]-(window_h/2), 0, -window_h)
    
    left_fit = np.polyfit(res_yvals, leftx, 2)
    left_fitx = left_fit[0]*yvals*yvals + left_fit[1]*yvals + left_fit[2]
    left_fitx = np.array(left_fitx, np.int32)
    
    right_fit = np.polyfit(res_yvals, rightx, 2)
    right_fitx = right_fit[0]*yvals*yvals + right_fit[1]*yvals + right_fit[2]
    right_fitx = np.array(right_fitx, np.int32)
    
    left_lane = np.array(list(zip(np.concatenate((left_fitx-window_w/2, left_fitx[::-1]+window_w/2), axis=0),
                                 np.concatenate((yvals, yvals[::-1]), axis=0))), np.int32)
    right_lane = np.array(list(zip(np.concatenate((right_fitx-window_w/2, right_fitx[::-1]+window_w/2), axis=0),
                                 np.concatenate((yvals, yvals[::-1]), axis=0))), np.int32)
    middle_marker = np.array(list(zip(np.concatenate((left_fitx+window_w/2, right_fitx[::-1]-window_w/2), axis=0),
                                 np.concatenate((yvals, yvals[::-1]), axis=0))), np.int32)
    
    
#     print("left_lane.shape = {}".format(right_lane))
    road = np.zeros_like(img)
#     road_bkg = np.zeros_like(img)
    
    cv2.fillPoly(road, [left_lane], color=[255, 0, 0])
    cv2.fillPoly(road, [right_lane], color=[0, 0, 255])
    cv2.fillPoly(road, [middle_marker], color=[0, 255, 0])
    cv2.fillPoly(road_bkg, [left_lane], color=[255, 255, 255])
    cv2.fillPoly(road_bkg, [right_lane], color=[255, 255, 255])
    
    road_warped = cv2.warpPerspective(road, Minv, img_size, flags=cv2.INTER_LINEAR)
    road_warped_bkg = cv2.warpPerspective(road_bkg, Minv, img_size, flags=cv2.INTER_LINEAR)
    
    base = cv2.addWeighted(img, 1.0, road_warped_bkg, -1.0, 0.0)
    result = cv2.addWeighted(img, 1.0, road_warped, 1.0, 0.0)
    
    
    camera_center = (left_fitx[-1] + right_fitx[-1])/2
    center_diff = (camera_center-warped.shape[1]/2)*xm_per_pix
    side_pos = 'left'
    if center_diff <= 0:
        side_pos = 'right'
        
        
    curve_fit_cr = np.polyfit(np.array(res_yvals, np.float32)*ym_per_pix,
                             np.array(leftx, np.float32)*xm_per_pix, 2)
    curverad = ((1 + \
                 (2*curve_fit_cr[0]*yvals[-1]*ym_per_pix + curve_fit_cr[1])**2)**1.5)/np.absolute(2*curve_fit_cr[0])
#     print("curverad = {}".format(curverad))
    cv2.putText(result, 'Radius of curvature {} m'.format(round(curverad,3)), 
                (50, 50), 
                cv2.FONT_HERSHEY_SIMPLEX, 
                1, 
                (255, 255, 255), 
                2)
    cv2.putText(result, 'Position: {} of center by {} m'.format(side_pos, round(center_diff,3)), 
                (50, 100), 
                cv2.FONT_HERSHEY_SIMPLEX, 
                1, 
                (255, 255, 255), 
                2)
    
    

    
    ofname = "./test_images/tracked{}.jpg".format(idx+1)  #Test index are 1 based
    cv2.imwrite(ofname, result)
    


/Users/hermansahota/miniconda3/envs/carnd-term1/lib/python3.5/site-packages/ipykernel/__main__.py:87: VisibleDeprecationWarning: using a non-integer number instead of an integer will result in an error in the future
/Users/hermansahota/miniconda3/envs/carnd-term1/lib/python3.5/site-packages/ipykernel/__main__.py:91: VisibleDeprecationWarning: using a non-integer number instead of an integer will result in an error in the future


In [ ]:



import numpy as np
import cv2
from matplotlib import pyplot as plt
import matplotlib.image as mpimg



def mag_thresh(image, sobel_kernel=3, thresh=(0, 255)):
    gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
    # Calculate directional gradient
    sobelx = cv2.Sobel(gray, cv2.CV_64F, 1, 0, ksize=sobel_kernel)
    sobely = cv2.Sobel(gray, cv2.CV_64F, 0, 1, ksize=sobel_kernel)
    sobel = np.sqrt(sobelx**2 + sobely**2)
    scaled_sobel = np.uint8(255 * sobel / np.max(sobel))
    mag_binary = np.zeros_like(scaled_sobel)
    mag_binary[(scaled_sobel >= thresh[0]) & (scaled_sobel <= thresh[1])] = 1
    return mag_binary

def dir_threshold(image, sobel_kernel=3, thresh=(0, np.pi/2)):
    gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
    # Calculate directional gradient
    sobelx = cv2.Sobel(gray, cv2.CV_64F, 1, 0, ksize=sobel_kernel)
    sobely = cv2.Sobel(gray, cv2.CV_64F, 0, 1, ksize=sobel_kernel)
    sobel_dir = np.arctan2(np.absolute(sobely), np.absolute(sobelx))
    dir_binary = np.zeros_like(sobel_dir)
    dir_binary[(sobel_dir >= thresh[0]) & (sobel_dir <= thresh[1])] = 1
    return dir_binary


def hls_select(img, thresh=(0, 255)):
    # 1) Convert to HLS color space
    hls = cv2.cvtColor(img, cv2.COLOR_RGB2HLS)
    # 2) Apply a threshold to the S channel
    s = hls[:,:,2]
    binary_output = np.zeros_like(s)
    binary_output[(s>thresh[0]) & (s<=thresh[1])] = 1
    # 3) Return a binary image of threshold result
    return binary_output

def pipeline(img, s_thresh=(170, 255), sx_thresh=(20, 100)):
    img = np.copy(img)
    # Convert to HSV color space and separate the V channel
    hsv = cv2.cvtColor(img, cv2.COLOR_RGB2HLS).astype(np.float)
    l_channel = hsv[:,:,1]
    s_channel = hsv[:,:,2]
    # Sobel x
    sobelx = cv2.Sobel(l_channel, cv2.CV_64F, 1, 0) # Take the derivative in x
    abs_sobelx = np.absolute(sobelx) # Absolute x derivative to accentuate lines away from horizontal
    scaled_sobel = np.uint8(255*abs_sobelx/np.max(abs_sobelx))
    
    # Threshold x gradient
    sxbinary = np.zeros_like(scaled_sobel)
    sxbinary[(scaled_sobel >= sx_thresh[0]) & (scaled_sobel <= sx_thresh[1])] = 1
    
    # Threshold color channel
    s_binary = np.zeros_like(s_channel)
    s_binary[(s_channel >= s_thresh[0]) & (s_channel <= s_thresh[1])] = 1
    # Stack each channel
    # Note color_binary[:, :, 0] is all 0s, effectively an all black image. It might
    # be beneficial to replace this channel with something else.
    color_binary = np.dstack(( np.zeros_like(sxbinary), sxbinary, s_binary))
    return color_binary


image = mpimg.imread("signs_vehicles_xygrad.jpg")

pipeline_binary = pipeline(image)

# Choose a Sobel kernel size
ksize = 3 # Choose a larger odd number to smooth gradient measurements

# Apply each of the thresholding functions
gradx = abs_sobel_thresh(image, orient='x', sobel_kernel=ksize, thresh=(20, 100))
grady = abs_sobel_thresh(image, orient='y', sobel_kernel=ksize, thresh=(20, 100))
mag_binary = mag_thresh(image, sobel_kernel=ksize, thresh=(20, 100))
dir_binary = dir_threshold(image, sobel_kernel=ksize, thresh=(0.7, 1.3))

hls_binary = hls_select(image, thresh=(90, 255))

combined = np.zeros_like(dir_binary)
combined[((gradx == 1) & (grady == 1)) | ((mag_binary == 1) & (dir_binary == 1))] = 1

f, (ax1, ax2) = plt.subplots(1, 2, figsize=(24, 9))
f.tight_layout()
gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
# ax1.imshow(gray, cmap='gray')
ax1.imshow(image)
ax1.set_title('Original Image', fontsize=50)
ax2.imshow(combined, cmap='gray')
ax2.set_title('Combined Gradient', fontsize=50)
plt.subplots_adjust(left=0., right=1, top=0.9, bottom=0.)
plt.show()